<a href="https://colab.research.google.com/github/gkianfar/TIHM-Dataset-Visualization/blob/main/tihm_featureExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Clone Github repo and set working directory path**

In [6]:
!git clone https://github.com/gkianfar/TIHM-Dataset-Visualization

fatal: destination path 'TIHM-Dataset-Visualization' already exists and is not an empty directory.


**Load packages**

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib import cm
import matplotlib.patches as mpatches
from collections.abc import Iterable
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
from matplotlib import cm
#from utils import *

%matplotlib inline
## Set colour palette
ibm_colorblind = ['#648FFF', '#FE6100', '#DC267F', '#785EF0', '#FFB000','#48A9A6']
sns.set_palette(ibm_colorblind)

**Functions**

In [8]:


def correct_col_type(df,col):
    raw_type = str(type(df[col].dtype)).split('.')[-1].split('\'')[0].lower()
    #print(col,raw_type)
    if 'object' in raw_type:
        if 'date' in col or 'timestamp' in col or 'datetime' in col:
            return pd.to_datetime(df[col])
        else:
            return df[col].astype('category')
    else:
        return df[col]


def gen_date_col(df, tcol):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df[tcol].dt.date
    return df





**Set Data and Fig folders paths**

In [9]:
## Please change the path with the path of your dataset
%cd /content/TIHM-Dataset-Visualization
DPATH = './Data'
SAVE_PATH = './Figs/'
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)



/content/TIHM-Dataset-Visualization


**Import Files**

In [10]:
### Labels
f = 'Labels.csv'
fpth = os.path.join(DPATH,f)
labels_df = pd.read_csv(fpth)
for col in labels_df.columns:
    labels_df[col] = correct_col_type(labels_df,col)
if 'date' in labels_df.columns:
    labels_df = labels_df.rename(columns={'date':'timestamp'})
    labels_df['timestamp'] = pd.to_datetime(labels_df['timestamp'])
    labels_df['date'] = pd.to_datetime(labels_df['timestamp'].dt.date)
if 'type' in labels_df.columns:
    labels_df = labels_df.rename(columns={'type':'label'})

In [11]:
### Activity

f = 'Activity.csv'
fpth = os.path.join(DPATH,f)
df = pd.read_csv(fpth)
for col in df.columns:
    df[col] = correct_col_type(df,col)
if 'date' in df.columns:
    activity_df = df.rename(columns={'date':'timestamp'})
    activity_df['timestamp'] = pd.to_datetime(activity_df['timestamp'])
    activity_df['date'] = pd.to_datetime(activity_df['timestamp'].dt.date)
    activity_df['time'] = activity_df['timestamp'].dt.time


activity_patients = np.unique(activity_df['patient_id'])

In [38]:
### Physiology
f = 'Physiology.csv'
fpth = os.path.join(DPATH,f)
df = pd.read_csv(fpth)
for col in df.columns:
    df[col] = correct_col_type(df,col)
if 'date' in df.columns:
    physiology_df = df.rename(columns={'date':'timestamp'})
    physiology_df['timestamp'] = pd.to_datetime(physiology_df['timestamp'])
    physiology_df['date'] = pd.to_datetime(physiology_df['timestamp'].dt.date)
#physiology_df = gen_date_col(df,tcol='timestamp')

physiology_patients = np.unique(physiology_df['patient_id'])
physiology_parameters = np.unique(df['device_type'])


In [39]:
# Sleep
f = 'Sleep.csv'
fpth = os.path.join(DPATH,f)
df = pd.read_csv(fpth)
for col in df.columns:
    df[col] = correct_col_type(df,col)
if 'date' in df.columns:
    sleep_df = df.rename(columns={'date':'timestamp'})
    sleep_df['timestamp'] = pd.to_datetime(sleep_df['timestamp'])
    sleep_df['date'] = pd.to_datetime(sleep_df['timestamp'].dt.date)
#physiology_df = gen_date_col(df,tcol='timestamp')

sleep_patients = np.unique(sleep_df['patient_id'])

In [40]:
interval = 6 # backward Time Window
agitation_df = labels_df[labels_df['label']=='Agitation'].copy()


agitation_patientID = agitation_df['patient_id'].tolist()
agitation_endTime = agitation_df['timestamp'].tolist()

agitation_startTime = pd.to_datetime(agitation_endTime) - pd.Timedelta(hours=interval)
agitation_startTime = agitation_startTime.tolist()

### patients that have agitation and the number of samples that each has
agitation_patients = np.unique(agitation_df['patient_id'])
each_patient_agitation_sample = agitation_df['patient_id'].value_counts()

# store the agitation days for each paient
agitation_days = {}
activity_days = {}
physiology_days = {}

for id in agitation_patients:
   agitation_days[id] = agitation_df['date'][agitation_df['patient_id'] == id].unique()
   activity_days[id] = activity_df['date'][activity_df['patient_id'] == id].unique()
   physiology_days[id] = physiology_df['date'][physiology_df['patient_id'] == id].unique()

agitation_dataset = []
# Iterate through each patient_id, start_time, and end_time
for index, (patient_id, start_time, end_time) in enumerate(zip(agitation_patientID, agitation_startTime, agitation_endTime)):
    print(f'{index}-user: {patient_id} - from {start_time} to {end_time}')
    # Filter the DataFrame for the current patient and time range
    filtered_df = activity_df[(activity_df['patient_id'] == patient_id) &
                            (activity_df['timestamp'] >= start_time) &
                            (activity_df['timestamp'] <= end_time)]

    # Store the results in the list
    agit_activity = filtered_df['location_name'].tolist()

    patient_agit_days = agitation_days[patient_id]
    patient_activity_days = activity_days[patient_id]
    pateint_non_agit_days = [date for date in patient_activity_days if date not in patient_agit_days]

    #print(f'nin agit len: {len(pateint_non_agit_days)}')

    # separate the corresponding non agitation intervals from normal days of each patient
    non_agit_activity = []
    for day in pateint_non_agit_days:
      if start_time.time() < end_time.time():
        filtered_df = activity_df[(activity_df['patient_id'] == patient_id) &
                                  (activity_df['date'] == day) &
                                  (activity_df['time'] >= start_time.time()) &
                                  (activity_df['time'] <= end_time.time())]
      else:
        raise Exception("Start time is after end time")


      filtered_activity = filtered_df['location_name'].tolist()
      non_agit_activity.append(filtered_activity)

    ### physiology parameters
    patient_physiology_days = physiology_days[patient_id]

    agit_params = []
    filtered_df = physiology_df[(physiology_df['patient_id']==patient_id) & (physiology_df['date']==end_time.date())]
    filtered_params = filtered_df['device_type'].tolist()
    filtered_unique_params = filtered_df['device_type'].unique().tolist()
    print(f'physio count {len(physiology_parameters)}')
    print(f'physio: {filtered_df}')
    print(end_time.date() in physiology_df[(physiology_df['patient_id']==patient_id)].date)

    for param in physiology_parameters:
        if (param in filtered_unique_params):
            agit_params.append((param,filtered_df['value'][filtered_df['device_type'] == param].mean()))
        else:
            agit_params.append((param,0))
    print(f'agit param {agit_params}')
    agitation_dataset.append([patient_id, start_time, end_time, agit_activity, non_agit_activity, agit_params])

0-user: 16f4b - from 2019-04-11 06:00:22 to 2019-04-11 12:00:22
physio count 8
physio: Empty DataFrame
Columns: [patient_id, timestamp, device_type, value, unit, date]
Index: []
False
agit param [('Body Temperature', 0), ('Body weight', 0), ('Diastolic blood pressure', 0), ('Heart rate', 0), ('O/E - muscle mass', 0), ('Skin Temperature', 0), ('Systolic blood pressure', 0), ('Total body water', 0)]
1-user: 16f4b - from 2019-04-14 06:00:07 to 2019-04-14 12:00:07


KeyboardInterrupt: 

In [33]:
for patient_id in agitation_patients:
  print(f'patient: {patient_id}')
  print(set(physiology_days[patient_id]) in set(agitation_days[patient_id]))

patient: 099bc
False
patient: 0cda9
False
patient: 0d5ef
False
patient: 16f4b
False
patient: 201d8
False
patient: 393cb
False
patient: 3fb61
False
patient: 46286
False
patient: 6b29b
False
patient: 73f7c
False
patient: 7db78
False
patient: 8a835
False
patient: 8d0d4
False
patient: 93c14
False
patient: 95899
False
patient: 96adf
False
patient: a2849
False
patient: b9d58
False
patient: c5031
False
patient: d263a
False
patient: d7a46
False
patient: d93d8
False
patient: e4959
False
patient: e87bd
False
patient: e8a78
False
patient: ec812
False
patient: eca1f
False


In [37]:
physiology_days

{'099bc': <DatetimeArray>
 ['2019-05-15 00:00:00', '2019-05-16 00:00:00', '2019-05-17 00:00:00',
  '2019-05-18 00:00:00', '2019-05-19 00:00:00', '2019-05-20 00:00:00',
  '2019-05-21 00:00:00', '2019-05-22 00:00:00', '2019-05-23 00:00:00',
  '2019-05-24 00:00:00', '2019-05-27 00:00:00', '2019-05-28 00:00:00',
  '2019-05-29 00:00:00', '2019-05-30 00:00:00', '2019-05-31 00:00:00',
  '2019-06-01 00:00:00', '2019-06-02 00:00:00', '2019-06-03 00:00:00',
  '2019-06-04 00:00:00', '2019-06-05 00:00:00', '2019-06-06 00:00:00',
  '2019-06-07 00:00:00', '2019-06-08 00:00:00', '2019-06-09 00:00:00',
  '2019-06-10 00:00:00', '2019-06-11 00:00:00', '2019-06-12 00:00:00',
  '2019-06-13 00:00:00', '2019-06-14 00:00:00', '2019-06-15 00:00:00',
  '2019-06-16 00:00:00', '2019-06-17 00:00:00', '2019-06-18 00:00:00',
  '2019-06-19 00:00:00', '2019-06-20 00:00:00', '2019-06-21 00:00:00',
  '2019-06-25 00:00:00', '2019-06-26 00:00:00', '2019-06-27 00:00:00',
  '2019-06-28 00:00:00', '2019-06-29 00:00:00', '20

In [ ]:
class AGITATION_DATASET():
  def __init__(self,agitation_dataset):
    num_samples = len(agitation_dataset)
    self.patient_id = [agitation_dataset[i][0] for i in range(num_samples) ]
    self.start_time = [agitation_dataset[i][1] for i in range(num_samples) ]
    self.end_time = [agitation_dataset[i][2] for i in range(num_samples) ]
    self.agitation_activity = [agitation_dataset[i][3] for i in range(num_samples) ]
    self.non_agitation = [agitation_dataset[i][4] for i in range(num_samples) ]
    self.init_df()

  def __len__(self):
    return len(self.patient_id)
  def init_df(self):
    self.feature_df = pd.DataFrame(np.column_stack((self.patient_id, self.start_time, self.end_time)),
                       columns=['patient_id', 'start_time', 'end_time'])
  def activity_change(self, inplace = True):
    activity_change = []
    for index in range(self.__len__()):
      dummy = self.agitation_activity[index]
      dummy = np.array(dummy)
      result = np.concatenate([[np.NaN], dummy[1:] != dummy[:-1]])
      changed = np.nansum(result)

      normal_activity_changes = []
      for arr in self.non_agitation[index]:
        arr = np.array(arr)
        result = np.concatenate([[np.NaN], arr[1:] != arr[:-1]])
        normal_activity_changes.append(np.nansum(result))

      # Calculate sample mean and standard deviation
      normal_activity_change_mean = np.mean(normal_activity_changes)
      normal_activity_change_std = np.std(normal_activity_changes, ddof=1)
      activity_change.append([changed,normal_activity_change_mean, normal_activity_change_std])
    if inplace:
      self.feature_df = pd.concat([self.feature_df,
        pd.DataFrame(activity_change, columns=['change_count', 'normal_change_mean','normal_change_std'])],axis=1)
    else:
      return pd.DataFrame(activity_change, columns=['change_count', 'normal_change_mean','normal_change_std'])

agitation_df = AGITATION_DATASET(agitation_dataset)



In [20]:
agitation_dataset[56][-1]

[('Body Temperature', 0),
 ('Body weight', 0),
 ('Diastolic blood pressure', 0),
 ('Heart rate', 0),
 ('O/E - muscle mass', 0),
 ('Skin Temperature', 0),
 ('Systolic blood pressure', 0),
 ('Total body water', 0)]

In [ ]:
agitation_df.activity_change()

In [ ]:
agitation_df.feature_df

,patient_id,start_time,end_time,change_count,normal_change_mean,normal_change_std
0,16f4b,2019-04-11 06:00:22,2019-04-11 12:00:22,187.0,79.782051,36.665086
1,16f4b,2019-04-14 06:00:07,2019-04-14 12:00:07,110.0,79.769231,36.700587
2,16f4b,2019-04-15 12:00:24,2019-04-15 18:00:24,133.0,86.397436,42.821690
3,16f4b,2019-04-16 12:00:38,2019-04-16 18:00:38,278.0,86.461538,42.803293
4,16f4b,2019-04-21 06:00:55,2019-04-21 12:00:55,154.0,79.858974,36.686359
...,...,...,...,...,...,...
130,0d5ef,2019-06-28 12:02:11,2019-06-28 18:02:11,203.0,104.947368,47.323468
131,d7a46,2019-06-28 12:02:21,2019-06-28 18:02:21,218.0,74.772727,59.840209
132,6b29b,2019-06-29 06:00:27,2019-06-29 12:00:27,137.0,85.375000,34.300956
133,95899,2019-06-29 06:01:20,2019-06-29 12:01:20,190.0,74.953488,24.551165


In [ ]:
# Mapping function for all categories in Location_name column

activity_locations_unique = activity_df['location_name'].unique()
activity_locations_map = {location: index for index, location in enumerate(activity_locations_unique)}

activity_change_count = []
activity_occurance_count = []
for filtered_activity in agitation_activity:

  dummy_filtered_activity = pd.Series(filtered_activity)
  activity_occurance_count.append(dummy_filtered_activity.value_counts())

  dummy_filtered_activity = dummy_filtered_activity.map(activity_locations_map)
  change_count = dummy_filtered_activity.diff().value_counts()

  if any(change_count.index == 0):
    unchanged_count = change_count[change_count.index == 0].values[0]
  else :
    unchanged_count = 0

  # Counting number of changes between locations by substracting the total number of unchanged locations from the max possible changes
  activity_change_count.append((dummy_filtered_activity.shape[0]-1)-unchanged_count)



In [ ]:
def filter_similar_intervals(x, end_time, interval=6):

    start_time = end_time - pd.Timedelta(hours=interval)
    if start_time <= end_time:
        # Interval does not span midnight
        return start_time <= x <= end_time
    else:
        # Interval spans midnight
        return x >= start_time or x <= end_time

# Given a time check if withing the future interval any agitation label exists (True: no agitation, False: existing agitation)
def filter_no_agitation_label(x, agitation_df, interval):
    end_time = x + pd.Timedelta(hours=interval)
    if agitation_df['timestamp'].between(x, end_time).any():
        return False
    else:
        return True

for patient_id, start_time, end_time in zip(agitation_patientID, agitation_startTime, agitation_endTime):
    # Filter the DataFrame for the current patient and time range
    filtered_df = activity_df[(activity_df['patient_id'] == patient_id)]
    filtered_df['similar_interval'] = filtered_df[filtered_df['timestamp'].apply(lambda x: filter_similar_intervals(x, end_time, interval))]


    # Store the results in the list
    filtered_activity = filtered_df['location_name'].tolist()

135

In [ ]:
labels_df['timestamp'].dt.time()

TypeError: 'Series' object is not callable